In [1]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
from datetime import datetime, date

In [2]:
from tqdm.notebook import tqdm

In [3]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [4]:
username = 'vigrose'
password_edw = 'Slytherin~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [5]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [6]:
ME_QUERY = \
    f"""
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND='Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [7]:
MED_QUERY = \
    """
    SELECT
    TOP_ID,
    EMPLOYER_ID,
    PARTY_ID
    FROM
    AMAEDW.MED_PROF
    WHERE
    THRU_DT IS NULL
    """
MED_PROF = pd.read_sql(con=AMAEDW, sql=MED_QUERY)

In [8]:
MPA_QUERY = \
    """
    SELECT
    MPA_CD,
    TOP_ID,
    EMPLOYER_ID,
    DESC
    FROM
    AMAEDW.MPA
    WHERE
    ACTIVE_IND='Y'
    """
MPA = pd.read_sql(con=AMAEDW, sql=MPA_QUERY)

In [9]:
SPEC_QUERY = \
    """
    SELECT
    M.PARTY_ID,
    M.SPEC_ID,
    M.PREFE_LVL
    FROM
    AMAEDW.MED_SPEC M
    """
SPEC = pd.read_sql(con=AMAEDW, sql=SPEC_QUERY)

In [10]:
AWARD_QUERY = \
    """
    SELECT
    PARTY_ID,
    EXPIRATION_DT
    FROM
    AMAEDW.PRA_CERTIFICATE
    WHERE
    PRA_STATUS_DESC='Approved'
    """
AWARD = pd.read_sql(con=AMAEDW, sql=AWARD_QUERY)

In [11]:
AWARD.EXPIRATION_DT = pd.to_datetime(AWARD.EXPIRATION_DT)

In [12]:
ov_universe = pd.merge(party_ids, ov_me, left_on='ME', right_on='medical_education_number')

In [13]:
wards = AWARD[AWARD.EXPIRATION_DT>datetime.today()].sort_values('EXPIRATION_DT').drop_duplicates('PARTY_ID', keep='last')

In [14]:
ALL = pd.merge(ov_universe, MED_PROF, on='PARTY_ID', how='left')
ALL = pd.merge(ALL, SPEC[SPEC.PREFE_LVL==1], on='PARTY_ID', how='left')
ALL = pd.merge(ALL, SPEC[SPEC.PREFE_LVL==2], on='PARTY_ID', suffixes = ['_PRIM','_SEC'], how='left')
ALL = pd.merge(ALL, MPA, on=['TOP_ID','EMPLOYER_ID'], how='left')
ALL = pd.merge(ALL, wards, on='PARTY_ID', how='left')

In [16]:
today = date.today()
types = ['Physician','Student','Resident']
elements = [
    {'data_element': 'TOP',
     'element':'TOP_ID',
    'null': 115.0},
    {'data_element': 'PE',
     'element':'EMPLOYER_ID',
    'null': 391.0},
    {'data_element': 'MPA',
     'element':'MPA_CD',
    'null': 'NCL'},
    {'data_element': 'Primary Specialty',
     'element':'SPEC_ID_PRIM',
    'null': 1883},
    {'data_element': 'Secondary Specialty',
     'element':'SPEC_ID_SEC',
    'null': 1883},
    {'data_element': 'PRA Award Flag',
     'element':'EXPIRATION_DT',
    'null': ''}
]
dict_list = []
for typo in types:
    total = len(ALL[ALL.type==typo])
    for element in elements:
        complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isna())])
        true_complete = len(ALL[(ALL.type==typo)&(ALL[element['element']]!=element['null'])&~(ALL[element['element']].isna())])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': total,
            'Date': today,
            'Measure':'Completeness'
        }
        dict_list.append(new_dict)

In [27]:
ALL[(ALL.type.isin(['Physician','Resident']))&(ALL.TOP_ID==115)&(ALL.EMPLOYER_ID==391)]

,PARTY_ID,ME,medical_education_number,type,TOP_ID,EMPLOYER_ID,SPEC_ID_PRIM,PREFE_LVL_PRIM,SPEC_ID_SEC,PREFE_LVL_SEC,MPA_CD,DESC,EXPIRATION_DT
382,1889101,00102921232,00102921232,Physician,115.0,391.0,1698.0,1.0,1883.0,2.0,NCL,Not Classified,NaT
622,1888978,00102851455,00102851455,Physician,115.0,391.0,1703.0,1.0,1883.0,2.0,NCL,Not Classified,NaT
690,1889180,00102961048,00102961048,Physician,115.0,391.0,1724.0,1.0,1883.0,2.0,NCL,Not Classified,NaT
691,1892621,00102961072,00102961072,Physician,115.0,391.0,1691.0,1.0,1883.0,2.0,NCL,Not Classified,NaT
765,1889242,00102991541,00102991541,Physician,115.0,391.0,1679.0,1.0,1883.0,2.0,NCL,Not Classified,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422747,12296778,93503100124,93503100124,Physician,115.0,391.0,1883.0,1.0,NaN,NaN,NCL,Not Classified,NaT
1422750,14377511,56002190023,56002190023,Physician,115.0,391.0,1883.0,1.0,NaN,NaN,NCL,Not Classified,NaT
1422751,6824748,24408170032,24408170032,Physician,115.0,391.0,1883.0,1.0,NaN,NaN,NCL,Not Classified,NaT
1422814,14035860,24104200016,24104200016,Physician,115.0,391.0,1883.0,1.0,NaN,NaN,NCL,Not Classified,NaT


In [28]:
ALL[(ALL.type.isin(['Physician','Resident']))]

,PARTY_ID,ME,medical_education_number,type,TOP_ID,EMPLOYER_ID,SPEC_ID_PRIM,PREFE_LVL_PRIM,SPEC_ID_SEC,PREFE_LVL_SEC,MPA_CD,DESC,EXPIRATION_DT
0,1889235,00102990919,00102990919,Physician,117.0,401.0,1639.0,1.0,1883.0,2.0,OFF,Office Based Practice,NaT
1,1904405,00102840976,00102840976,Physician,117.0,418.0,1755.0,1.0,1724.0,2.0,OFF,Office Based Practice,NaT
2,1892959,00106930210,00106930210,Physician,117.0,424.0,1691.0,1.0,1883.0,2.0,OFF,Office Based Practice,NaT
3,1903519,00102540336,00102540336,Physician,113.0,418.0,1763.0,1.0,1883.0,2.0,INA,Inactive,NaT
4,1903525,00102550285,00102550285,Physician,113.0,391.0,1672.0,1.0,1883.0,2.0,INA,Inactive,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422840,13588004,79601210181,79601210181,Resident,116.0,402.0,1724.0,1.0,1883.0,2.0,HPR,Hospital Based Residents - All Years,NaT
1422841,6525524,13002181520,13002181520,Resident,116.0,402.0,1724.0,1.0,1883.0,2.0,HPR,Hospital Based Residents - All Years,NaT
1422842,10426769,49623190071,49623190071,Resident,116.0,402.0,1724.0,1.0,1883.0,2.0,HPR,Hospital Based Residents - All Years,NaT
1422843,11207668,66102200315,66102200315,Resident,116.0,402.0,1724.0,1.0,1883.0,2.0,HPR,Hospital Based Residents - All Years,NaT


In [30]:
119416/1336668*100

8.933856425080872

In [15]:
pd.DataFrame(dict_list).to_csv(f'../../Data/Measurement/Practice_Completeness_{str(date.today())}.csv', index=False)

In [14]:
ALL[ALL.type=='Student'].TOP_ID.unique()

array([116.,  nan, 115.])

In [16]:
ALL[ALL.type=='Student'].EMPLOYER_ID.unique()

array([402.,  nan, 407.])

In [18]:
spec_codes = pd.read_csv('../../Data/Measurement/SPECIALTY.csv')

In [27]:
spick = list(spec_codes.SPEC_ID)

In [32]:
ALL[~(ALL.SPEC_ID_PRIM.isin(spick))&(~ALL.SPEC_ID_PRIM.isna())]

,PARTY_ID,ME,medical_education_number,type,TOP_ID,EMPLOYER_ID,SPEC_ID_PRIM,PREFE_LVL_PRIM,SPEC_ID_SEC,PREFE_LVL_SEC


In [31]:
ppd = pd.read_csv('../../Data/PPD/ppd_analysis_file.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,24,48,57,64,65,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
hmm = ppd[ppd.duplicated('POLO_COMM_ID', keep=False)].sort_values('POLO_COMM_ID')[['POLO_COMM_ID', 'ME', 'PE_CD']]

In [34]:
hmm = hmm[~hmm.POLO_COMM_ID.isna()&~(hmm.PE_CD.isna())]

In [35]:
pe_cds = pd.read_csv('../../Data/Measurement/EMPLOYMENT.csv')

In [36]:
hmm = hmm.to_dict('records')

In [37]:
unique_polos = []
polo_info = {}
for row in tqdm(hmm):
    if row['POLO_COMM_ID'] not in unique_polos:
        unique_polos.append(row['POLO_COMM_ID'])
        polo_info[row['POLO_COMM_ID']] = {row['PE_CD'] : 1}
    elif row['PE_CD'] in polo_info[row['POLO_COMM_ID']].keys():
        polo_info[row['POLO_COMM_ID']][row['PE_CD']]+=1
    else:
        polo_info[row['POLO_COMM_ID']][row['PE_CD']] = 1

In [38]:
def get_key(val, my_dict):
    keys = []
    for key, value in my_dict.items():
         if val == value:
            keys.append(key)
    return keys

In [71]:
address_dict_list = []
for address in tqdm(unique_polos):
    new_dict = {}
    new_dict['POLO_COMM_ID'] = address
    key_list = list(polo_info[address].keys())
    if len(key_list)==1:
        best_code = key_list
    else:
        val_list = list(polo_info[address].values())
        highest = max(val_list)
        best_code = get_key(highest, polo_info[address])
    if 110 in best_code:
        best_code.remove(110)
    if best_code!=[]:
        new_dict['PE_CD'] = best_code
        address_dict_list.append(new_dict)

In [72]:
address_dict_list

[{'POLO_COMM_ID': 713750.0, 'PE_CD': [50]},
 {'POLO_COMM_ID': 713768.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 714449.0, 'PE_CD': [30, 50]},
 {'POLO_COMM_ID': 714609.0, 'PE_CD': [50, 30]},
 {'POLO_COMM_ID': 714886.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 714902.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 714952.0, 'PE_CD': [11, 30]},
 {'POLO_COMM_ID': 715391.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 715770.0, 'PE_CD': [30, 63]},
 {'POLO_COMM_ID': 716150.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 716153.0, 'PE_CD': [11]},
 {'POLO_COMM_ID': 716301.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 716746.0, 'PE_CD': [50]},
 {'POLO_COMM_ID': 717581.0, 'PE_CD': [63]},
 {'POLO_COMM_ID': 718250.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 718871.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 719567.0, 'PE_CD': [13, 30]},
 {'POLO_COMM_ID': 719675.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 719922.0, 'PE_CD': [50, 30, 63]},
 {'POLO_COMM_ID': 720058.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 720386.0, 'PE_CD': [30]},
 {'POLO_COMM_ID': 720395.0, 'PE_CD': [11, 21]},


In [73]:
best_pe_codes = pd.DataFrame(address_dict_list)

In [85]:
DET = pd.merge(best_pe_codes, ppd[['DESC','ME','POLO_COMM_ID','PE_CD','TOP_CD']], on='POLO_COMM_ID', suffixes = ['_GOOD','_ACTUAL']).drop_duplicates('ME')

KeyError: "['DESC'] not in index"

In [75]:
PE_LIST = []
for row in DET.itertuples():
    PE_GOOD = row.PE_CD_ACTUAL in row.PE_CD_GOOD
    PE_LIST.append(PE_GOOD)

In [76]:
DET['MATCH'] = PE_LIST

In [77]:
DET[(DET.MATCH == False)&(DET.PE_CD_ACTUAL == 110)&(DET.TOP_CD==20)]

,POLO_COMM_ID,PE_CD_GOOD,ME,PE_CD_ACTUAL,TOP_CD,MATCH
38,717581.0,[63],1108170626,110,20,False
57,719922.0,"[50, 30, 63]",512901387,110,20,False
69,720671.0,"[30, 63]",301060441,110,20,False
95,723155.0,[11],502020749,110,20,False
101,723746.0,[63],1103900910,110,20,False
...,...,...,...,...,...,...
40581,28651282.0,[30],1001940500,110,20,False
40587,28654244.0,[30],1002141074,110,20,False
40596,28661604.0,[30],512150381,110,20,False
40613,28666881.0,"[101, 63]",506010241,110,20,False


In [67]:
DET[(DET.MATCH == False)].to_csv('../../Data/Measurement/potential_pe_MORE.csv', index=False)

In [78]:
DET[DET.MATCH == False].groupby('PE_CD_ACTUAL').count()

,POLO_COMM_ID,PE_CD_GOOD,ME,TOP_CD,MATCH
PE_CD_ACTUAL,,,,,
11,422,422,422,422,422
13,129,129,129,129,129
21,95,95,95,95,95
22,27,27,27,27,27
30,641,641,641,641,641
35,175,175,175,175,175
40,287,287,287,287,287
50,838,838,838,838,838
63,1257,1257,1257,1257,1257


In [82]:
present_employment

NameError: name 'present_employment' is not defined

In [80]:
DET[DET.MATCH]

,POLO_COMM_ID,PE_CD_GOOD,ME,PE_CD_ACTUAL,TOP_CD,MATCH
2,713750.0,[50],1002101471,50,20,True
4,713750.0,[50],1002141970,50,20,True
6,713750.0,[50],1275160632,50,20,True
7,713768.0,[30],102150775,30,20,True
8,713768.0,[30],102980328,30,20,True
...,...,...,...,...,...,...
40691,28755278.0,[50],1205180334,50,12,True
40692,28755278.0,[50],1205190216,50,12,True
40693,28755278.0,[50],1221200914,50,12,True
40694,28755314.0,[50],401211624,50,12,True


In [38]:
import math

In [ ]:
address_dict_list_2 = []
for address in tqdm(unique_polos):
    new_dict = {}
    new_dict['POLO_COMM_ID'] = address
    key_list = list(polo_info[address].keys())
    if len(key_list)==1:
        best_code = key_list
    else:
        val_list = list(polo_info[address].values())
        highest = max(val_list)
        best_code = get_key(highest, polo_info[address])
    if 110 in best_code:
        best_code.remove(110)
    new_dict['PE_CD'] = best_code
    address_dict_list_2.append(new_dict)

In [1]:
address_dict_list_2

NameError: name 'address_dict_list_2' is not defined

In [64]:
pe_cds

,EMPLOYER_ID,EMPLOYER_CD,DESC,SRC_SYS,INSERT_TMSTP,INSERT_USERID,UPDATE_TMSTP,UPDATE_USERID,ACTIVE_IND
0,388,62,City/County/State Government Other,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
1,389,10,Self Employed Solo,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
2,390,90,Federal Government Other Than Hospital,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
3,391,110,No Classification,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
4,392,0,Student,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
5,393,40,Medical School,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
6,394,61,City/County/State Government Hospital,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
7,395,72,State Goverment Other,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
8,396,92,Federal Government-Other Than Hosp/Navy,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y
9,397,95,Federal Government-Other Than Hosp/Vet Admin,MASTERFILE,16DEC2005:03:34:54.000000,ETL_LOAD,16DEC2005:14:53:45.000000,ETL_LOAD,Y


In [3]:
import numpy as np

In [12]:
1-(np.random.random_sample()*.2)

0.8339198760913384

In [19]:
np.random.uniform(0.9, 1.1, 6)

array([0.93428987, 1.05173982, 1.05893071, 1.01645534, 1.08556043,
       0.97901888])

In [41]:
ALL[(ALL.type=='Student')&(~ALL.EMPLOYER_ID.isna())]

,PARTY_ID,ME,medical_education_number,type,TOP_ID,EMPLOYER_ID,SPEC_ID_PRIM,PREFE_LVL_PRIM,SPEC_ID_SEC,PREFE_LVL_SEC,MPA_CD,DESC,EXPIRATION_DT
75986,6166127,04501151632,04501151632,Student,116.0,402.0,NaN,NaN,NaN,NaN,HPR,Hospital Based Residents - All Years,NaT
1033453,5164678,04114121113,04114121113,Student,116.0,402.0,NaN,NaN,NaN,NaN,HPR,Hospital Based Residents - All Years,NaT
1062896,5598448,00511131146,00511131146,Student,116.0,402.0,NaN,NaN,NaN,NaN,HPR,Hospital Based Residents - All Years,NaT
1089585,5875176,04113140238,04113140238,Student,116.0,402.0,NaN,NaN,NaN,NaN,HPR,Hospital Based Residents - All Years,NaT


In [45]:
ALL[(ALL.type=='Student')&(~ALL.SPEC_ID_SEC.isna())]

,PARTY_ID,ME,medical_education_number,type,TOP_ID,EMPLOYER_ID,SPEC_ID_PRIM,PREFE_LVL_PRIM,SPEC_ID_SEC,PREFE_LVL_SEC,MPA_CD,DESC,EXPIRATION_DT
75775,6165903,04203150585,04203150585,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
166665,6441002,02507161521,02507161521,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
167022,6441413,02604161544,02604161544,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
171223,6445908,04201160605,04201160605,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
280728,6736200,04201170384,04201170384,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
280941,6736433,04204170563,04204170563,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
281548,6737115,04804171458,04804171458,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
335163,6737611,04814171618,04814171618,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
1056761,5555126,02512130537,02512130537,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT
1082085,5829321,03548180014,03548180014,Student,NaN,NaN,NaN,NaN,1883.0,2.0,NaN,NaN,NaT


In [84]:
DET

,POLO_COMM_ID,PE_CD_GOOD,ME,PE_CD_ACTUAL,TOP_CD,MATCH
0,713750.0,[50],511100186,30,20,False
1,713750.0,[50],801810055,30,20,False
2,713750.0,[50],1002101471,50,20,True
3,713750.0,[50],1002132041,63,20,False
4,713750.0,[50],1002141970,50,20,True
...,...,...,...,...,...,...
40691,28755278.0,[50],1205180334,50,12,True
40692,28755278.0,[50],1205190216,50,12,True
40693,28755278.0,[50],1221200914,50,12,True
40694,28755314.0,[50],401211624,50,12,True
